In [1]:
import platform
import os
import numpy as np
import keras
from keras import layers

if platform.machine() == 'x86_64':
    os.chdir("/home/sakshmeno/Documents/GitHub/C-RNN-approach")
elif platform.machine() == 'arm64':
    os.chdir("/Users/saksh.menon/Documents/GitHub/C-RNN-approach")

from Utils.dataframe_gen import *
from Utils.OneHot_gen import *
from Utils.TFIDF_gen import *
from Utils.word2vec_gen import *
from Utils.vector_gen import *



2024-02-29 14:52:53.459322: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-29 14:52:53.502524: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 14:52:53.502552: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 14:52:53.503669: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-29 14:52:53.509503: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-29 14:52:53.509911: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
original_df = dataframe_init()
word2vec_df = original_df.copy()
word2vec_df = word2vec_init(word2vec_df)

/home/sakshmeno/Documents/GitHub/C-RNN-approach/Utils/word2vec_gen.py:60: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['Lines'][vector[0]] = np.array(vector[1]).astype(dtype="float32")


In [3]:
vectors = vec_split(word2vec_df)
x_train, x_test, y_train, y_test = tensors = tensor_gen(vectors)

In [12]:
model = keras.Sequential()

model.add(layers.Input((218, 1)))
model.add(layers.SimpleRNN(256, return_sequences=True, dropout=0.4))
model.add(layers.SimpleRNN(256, return_sequences=True, dropout=0.4))
model.add(layers.SimpleRNN(256, dropout=0.4))
model.add(layers.Dense(256, activation=keras.activations.relu)) #kernel_regularizer=keras.regularizers.l2(0.01)
model.add(layers.Dense(256, activation=keras.activations.relu ))
model.add(layers.Dense(256, activation=keras.activations.relu ))
model.add(layers.Dense(2, activation = keras.activations.softmax))

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss=keras.losses.BinaryFocalCrossentropy(apply_class_balancing=True,gamma=2),
              metrics=["accuracy"], )
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_9 (SimpleRNN)    (None, 218, 256)          66048     
                                                                 
 simple_rnn_10 (SimpleRNN)   (None, 218, 256)          131328    
                                                                 
 simple_rnn_11 (SimpleRNN)   (None, 256)               131328    
                                                                 
 dense_12 (Dense)            (None, 256)               65792     
                                                                 
 dense_13 (Dense)            (None, 256)               65792     
                                                                 
 dense_14 (Dense)            (None, 256)               65792     
                                                                 
 dense_15 (Dense)            (None, 2)                

In [11]:
history = model.fit(x_train, y_train, epochs=10, batch_size=32)

Epoch 1/10


ValueError: in user code:

    File "/home/sakshmeno/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/home/sakshmeno/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/sakshmeno/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/home/sakshmeno/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/sakshmeno/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/home/sakshmeno/.local/lib/python3.10/site-packages/keras/src/engine/compile_utils.py", line 277, in

In [8]:
y_pred = model.predict(x_test)

33/33 [==============================] - 3s 81ms/step


In [9]:
list(y_pred)

[array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 0.08374306], dtype=float32),
 array([0.91625696, 

In [ ]:
(y_test[1040])

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>